<a href="https://colab.research.google.com/github/shreyasat27/pennylane-27524/blob/main/Introduction_to_mid_circuit_measurements.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane
import pennylane as qml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 28.0 MB/s eta 0:00:00


Measuring a single qubit

In [ ]:
#define a device
dev =qml.device("default.qubit")

#define node
@qml.qnode(dev)
def before():
  qml.Hadamard(0) #creates the plus state
  return qml.expval(qml.X(0)), qml.expval(qml.Z(0))

b = before()

print(f"Expectation values before any measurement: {b[0]:.1f},{b[1]:.1f}")

Expectation values before any measurement: 1.0,0.0


In [ ]:
@qml.qnode(dev)

def after():
  qml.Hadamard(0) #create a plus state
  qml.measure(0)
  return qml.expval(qml.X(0)), qml.expval(qml.Z(0))

a = after()
print(f"Expectation value after the measurement: {a[0]:.1f}, {a[1]:.1f}")



Expectation value after the measurement: 0.0, 0.0


In [ ]:
#post selection

@qml.qnode(dev)

def after():
  qml.Hadamard(0) #plus state
  qml.measure(0, postselect=0) #measure and only accept 1 as outcome
  return qml.expval(qml.X(0)), qml.expval(qml.Z(0))

a = after()
print(f"Expectation value after the measurement: {a[0]:.1f}, {a[1]:.1f}")

Expectation value after the measurement: 0.0, 1.0


Measuring a Bell pair

In [ ]:
def bell_pair_prepration(**kwargs):
    qml.Hadamard(0)
    qml.CNOT([0,1]) #create a bell pair
    qml.measure(0,**kwargs) #here we are measuring the first qubit, using keyword arguments



In [ ]:
@qml.qnode(dev)
def bell_pair(postselect):
  bell_pair_prepration(postselect=postselect)
  return qml.purity([0,1]), qml.vn_entropy(0)



comapring the ppurities and von Neumann entropies of the Bell state after measurement:

In [ ]:
without_ps = bell_pair(None)
with_ps = bell_pair(1)
print(f"                     |without ps | with ps ")
print(f"Purity               |    {without_ps[0]:.1f}  | {with_ps[0]:.1f}")
print(f"Entanglement entropy |    {without_ps[1]:.2f} | {with_ps[1]:.1f}")



                     |without ps | with ps 
Purity               |    0.5  | 1.0
Entanglement entropy |    0.69 | -0.0


Qubit Reset

In [ ]:
@qml.qnode(dev)
def bell_pair_with_reset(reset):
  bell_pair_prepration(reset=reset)
  return qml.expval(qml.Z(0)), qml.expval(qml.Z(1)), qml.expval(qml.Z(0) @ qml.Z(1))
no_reset = bell_pair_with_reset(reset=False)
reset =bell_pair_with_reset(reset=True)

print(f"              | <Z₀> | <Z₁> | <Z₀Z₁> ")
print(f"Without reset |  {no_reset[0]:.1f} |  {no_reset[1]:.1f} |   {no_reset[2]:.1f}")
print(f"With reset    |  {reset[0]:.1f} |  {reset[1]:.1f} |   {reset[2]:.1f}")






              | <Z₀> | <Z₁> | <Z₀Z₁> 
Without reset |  0.0 |  0.0 |   1.0
With reset    |  1.0 |  0.0 |   0.0


Dynamically controlling a quantum circuit

1. T-gadget (a technique related to quantum teleportation)

In [ ]:
import numpy as np

magic_state = np.array([1, np.exp(1j * np.pi / 4)]) / np.sqrt(2)

def t_gadget(wire, aux_wire):
    qml.QubitStateVector(magic_state, aux_wire)
    qml.CNOT([wire, aux_wire])
    mcm = qml.measure(aux_wire, reset=True)  # Resetting disentangles aux qubit
    qml.cond(mcm, qml.S)(wire)  # Apply qml.S(wire) if mcm was 1

In [ ]:
@qml.qnode(dev)
def test_t_gadget(init_state):
    qml.Hadamard(0)  # Create |+> state
    if init_state == "-":
        qml.Z(0)  # Flip to |-> state

    t_gadget(0, 1)  # Apply T-gadget
    qml.adjoint(qml.T)(0)  # Apply T^† to undo the gadget

    return qml.expval(qml.X(0))

print(f"<X₀> with initial state |+>: {test_t_gadget('+'):4.1f}")
print(f"<X₀> with initial state |->: {test_t_gadget('-'):4.1f}")

<X₀> with initial state |+>:  1.0
<X₀> with initial state |->: -1.0
